In [ ]:
!mkdir docker

In [ ]:
%%writefile docker/Dockerfile

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 fbprophet sagemaker-training
COPY train.py /opt/ml/code/train.py
ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3", "/opt/ml/code/train.py"]

In [ ]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'sagemaker-processing-container'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [ ]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository docker

In [ ]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

In [ ]:
!aws ecr create-repository --repository-name $ecr_repository

In [ ]:
!docker tag {ecr_repository + tag} $processing_repository_uri

In [ ]:
!docker push $processing_repository_uri